# 1. Configuración del Entorno y Librerías
En esta sección instalamos el SDK oficial de Google Gen AI (`google-genai`).
Es importante usar la bandera `-U` para asegurar que tenemos la última versión compatible con los modelos más recientes (Gemini 2.5/1.5).
También importamos las bibliotecas necesarias para manipular archivos (`os`, `json`), manejar el tiempo (`time`), generar aleatoriedad (`random`) y conectar con Google Drive.

In [34]:
# Instalamos la librería oficial de Google Gen AI (versión actualizada)
# El '!' indica que es un comando de sistema en Colab, no Python puro.
!pip install -U google-genai

# --- Importación de módulos estándar ---
import os           # Para interactuar con el sistema operativo (crear carpetas, rutas)
import sys          # Para acceder a variables y funciones del sistema
import time         # Para pausar la ejecución (sleep) y manejar tiempos
import json         # Para parsear y generar archivos en formato JSON
import random       # Para añadir aleatoriedad (jitter) en los tiempos de espera

# --- Importación del SDK de Google ---
from google import genai        # Cliente principal de la API
from google.genai import types  # Tipos de datos para configuración estricta

# --- Importaciones específicas de Google Colab ---
from google.colab import drive  # Para montar tu Google Drive en la notebook
from google.colab import files  # Para descargar archivos automáticamente al PC

# 1.1. Diagnóstico de Capacidades (Opcional)
Este script es una herramienta de **descubrimiento**. Conecta con la API de Google y solicita el catálogo de modelos disponibles para el tipo de cuenta del usuario
Es útil ejecutarlo para verificar:
1.  Que la API Key es válida.
2.  A qué versiones específicas del modelo tenemos acceso (ej. `gemini-2.5-flash`, `gemini-1.5-pro`).
3.  Evitar errores de "Modelo no encontrado" copiando el ID exacto de la lista resultante.

In [35]:
# Pega tu API KEY aquí
API_KEY = "--------------"

client = genai.Client(api_key=API_KEY)

print("--- Modelos Disponibles ---")
try:
    # Solicitamos la lista de modelos que soportan 'generateContent'
    for model in client.models.list():
        # Filtramos para ver solo los Gemini actuales
        if "gemini" in model.name:
            print(f"Nombre: {model.name}")
            print(f"Display Name: {model.display_name}")
            print("-" * 20)

except Exception as e:
    print(f"Error al listar modelos: {e}")

--- Modelos Disponibles ---
Nombre: models/gemini-2.5-flash
Display Name: Gemini 2.5 Flash
--------------------
Nombre: models/gemini-2.5-pro
Display Name: Gemini 2.5 Pro
--------------------
Nombre: models/gemini-2.0-flash
Display Name: Gemini 2.0 Flash
--------------------
Nombre: models/gemini-2.0-flash-001
Display Name: Gemini 2.0 Flash 001
--------------------
Nombre: models/gemini-2.0-flash-exp-image-generation
Display Name: Gemini 2.0 Flash (Image Generation) Experimental
--------------------
Nombre: models/gemini-2.0-flash-lite-001
Display Name: Gemini 2.0 Flash-Lite 001
--------------------
Nombre: models/gemini-2.0-flash-lite
Display Name: Gemini 2.0 Flash-Lite
--------------------
Nombre: models/gemini-exp-1206
Display Name: Gemini Experimental 1206
--------------------
Nombre: models/gemini-2.5-flash-preview-tts
Display Name: Gemini 2.5 Flash Preview TTS
--------------------
Nombre: models/gemini-2.5-pro-preview-tts
Display Name: Gemini 2.5 Pro Preview TTS
-----------------

# 2. Configuración de Credenciales y Almacenamiento
Aquí definimos las constantes globales del proyecto.
* **API Key:** La llave de acceso a Gemini.
* **Modelo:** Seleccionamos `gemini-2.5-flash` por su velocidad
* **Google Drive:** Configuramos la ruta donde se guardarán los resultados (`/content/drive/MyDrive/...`) para asegurar persistencia de datos.

In [36]:

# --- 1. CONFIGURACIÓN E INICIALIZACIÓN ---

# Modelo
MODEL_ID = "gemini-2.5-flash"

# Configuración de carpetas en Drive
DRIVE_MOUNT_PATH = '/content/drive'
PROJECT_FOLDER = '/content/drive/MyDrive/PruebaTecnicaNetec/Gemini_Logs_Project'

def setup_environment():
    """
    Monta Google Drive y prepara el directorio de trabajo.
    """
    print("--- Montando Google Drive ---")
    drive.mount(DRIVE_MOUNT_PATH)

    # Crear carpeta si no existe
    if not os.path.exists(PROJECT_FOLDER):
        print(f"Creando carpeta del proyecto en: {PROJECT_FOLDER}")
        os.makedirs(PROJECT_FOLDER)
    else:
        print(f"Carpeta del proyecto encontrada: {PROJECT_FOLDER}")

    # Verificar si existe logs.txt en Drive, si no, crear uno de prueba
    input_path = os.path.join(PROJECT_FOLDER, "logs.txt")
    if not os.path.exists(input_path):
        print("No se encontró logs.txt en la carpeta. Creando uno de prueba...")
        dummy_logs = """[2025-10-20 17:40:29] INFO: Error al procesar la solicitud del agente.
[2025-10-20 14:10:04] ERROR: Service restarted due to failure.
[2025-10-20 17:32:13] WARNING: Session closed unexpectedly.
[2025-10-20 18:00:00] INFO: System check OK.
[2025-10-20 19:00:00] ERROR: Database timeout."""
        with open(input_path, "w", encoding="utf-8") as f:
            f.write(dummy_logs)

    return input_path

# 3. Procesamiento ETL con Tolerancia a Fallos
La función (`analyze_logs`) es el núcleo del script. Implementa:
1.  **Conexión a la API:** Instancia el cliente de Gemini.
2.  **Backoff Exponencial:** Si ocurre un error 429 (Cuota excedida), espera tiempos crecientes (2s, 4s, 8s...).
3.  **Circuit Breaker:** Si 3 logs fallan consecutivamente, el script se detiene para proteger la cuenta y guarda el progreso.
4.  **Descarga Automática:** Al finalizar, descarga el JSON resultante a tu computadora local.


In [37]:
def analyze_logs(input_file_path: str):
    """
    Función principal que lee logs, consulta al LLM y guarda resultados JSON.
    Incluye lógica de 'Circuit Breaker' para detenerse si la API falla mucho.
    """

    # Definimos la ruta de salida combinando la carpeta del proyecto y el nombre del archivo
    output_file_path = os.path.join(PROJECT_FOLDER, "output.json")

    # --- 1. Instanciación del Cliente ---
    try:
        # Creamos el cliente usando la API KEY configurada
        client = genai.Client(api_key=API_KEY)
    except Exception as e:
        # Si falla la autenticación, terminamos la función
        print(f"❌ Error crítico al iniciar el cliente: {e}")
        return

    print(f"--- 🤖 Iniciando análisis con {MODEL_ID} ---")
    print(f"📄 Leyendo archivo desde: {input_file_path}")

    # Lista vacía donde acumularemos los resultados procesados
    results = []

    try:
        # --- 2. Lectura del Archivo ---
        with open(input_file_path, 'r', encoding='utf-8') as f:
            # Leemos todas las líneas y eliminamos espacios en blanco al inicio/final
            # 'if line.strip()' sirve para ignorar líneas totalmente vacías
            logs = [line.strip() for line in f if line.strip()]

        total = len(logs)
        print(f"📊 Volumen de datos: {total} registros detectados.")

        # Variables para el control de flujo (Circuit Breaker)
        consecutive_failures = 0        # Contador de fallos seguidos
        MAX_CONSECUTIVE_FAILURES = 3    # Límite máximo de fallos permitidos antes de abortar

        # --- 3. Bucle de Procesamiento ---
        for i, log_text in enumerate(logs):

            # VERIFICACIÓN DE SEGURIDAD (Circuit Breaker)
            # Si hemos fallado 3 veces seguidas, asumimos que la API está caída o cuota agotada
            if consecutive_failures >= MAX_CONSECUTIVE_FAILURES:
                print("\n" + "⛔" * 20)
                print(f"DETENCIÓN DE EMERGENCIA: {MAX_CONSECUTIVE_FAILURES} fallos consecutivos.")
                print("Guardando progreso parcial y terminando ejecución.")
                print("⛔" * 20 + "\n")
                break # Rompemos el bucle for principal

            # Configuración de reintentos para EL MISMO log actual
            max_retries = 5  # Intentaremos hasta 5 veces procesar este log específico
            attempt = 0      # Intento actual
            success = False  # Bandera de éxito

            # Bucle While: Se repite mientras no tengamos éxito y no agotemos intentos
            while attempt < max_retries and not success:
                try:
                    # ---- 4. Configuración de Inferencia  ------
                    # response_mime_type="application/json" fuerza respuesta estructurada
                    generation_config = types.GenerateContentConfig(
                        temperature=0.0, # Temperatura 0 para máxima precisión y determinismo
                        response_mime_type="application/json",
                        system_instruction=(
                            "Eres un analista de sistemas experto. Tu tarea es clasificar logs."
                            "Responde ÚNICAMENTE con una lista JSON de strings (etiquetas). "
                            "Ejemplo: [\"Error\", \"Database\", \"Timeout\"]"
                        )
                    )

                    # ------ 5. Llamada a la API (Inferencia) ------
                    response = client.models.generate_content(
                        model=MODEL_ID,
                        contents=f"Clasifica: {log_text}",
                        config=generation_config
                    )

                    # Parseo de la respuesta (Texto -> JSON Objeto)
                    tags = json.loads(response.text)

                    # Agregamos el resultado a nuestra lista maestra
                    results.append({
                        "log_id": i + 1,
                        "texto": log_text,
                        "etiquetas": tags
                    })

                    # Feedback en consola para el usuario
                    print(f"✅ [{i+1}/{total}] Procesado: {tags}")

                    # ÉXITO: Actualizamos banderas
                    success = True
                    consecutive_failures = 0 # Reiniciamos contador de fallos globales

                    # Pausa de cortesía (Rate Limiting)
                    # 4 segundos es seguro para la capa gratuita (aprox 15 req/min)
                    time.sleep(4.0)

                except Exception as e:
                    # Manejo de Errores
                    error_str = str(e)

                    # Si el error es código 429 (Too Many Requests / Resource Exhausted)
                    if "429" in error_str or "RESOURCE_EXHAUSTED" in error_str:
                        # Cálculo de espera exponencial (Exponential Backoff)
                        # Formula: 2 elevado al intento + un poco de azar + 5 seg base
                        wait_time = (2 ** attempt) + random.uniform(1, 3) + 5
                        print(f"⚠️ [Log {i+1}] Cuota excedida (Intento {attempt+1}/{max_retries}). Esperando {wait_time:.1f}s...")

                        # Pausamos el script antes de reintentar
                        time.sleep(wait_time)
                        attempt += 1 # Incrementamos contador de intentos local
                    else:
                        # Si es otro error (ej. error interno del servidor), no reintentamos
                        print(f"❌ Error irrecuperable en log {i+1}: {e}")
                        results.append({"log_id": i+1, "error": str(e)})
                        break # Salimos del while local

            # Si salimos del while y NO hubo éxito (se agotaron los 5 intentos)
            if not success:
                print(f"❌ Se abandonó el log {i+1} tras {max_retries} intentos.")
                consecutive_failures += 1 # Sumamos un fallo a la cuenta global
                results.append({"log_id": i+1, "texto": log_text, "error": "Max Retries Exceeded"})

        # --- 4. Guardado y Descarga ---
        print(f"💾 Guardando resultados en Drive: {output_file_path}")
        # Escribimos el archivo final en JSON
        with open(output_file_path, 'w', encoding='utf-8') as f:
            json.dump(results, f, indent=4, ensure_ascii=False)

        print(f"🎉 Proceso finalizado.")

        # Intentamos descargar el archivo automáticamente al PC local
        print("⬇️ Iniciando descarga local automática...")
        try:
            files.download(output_file_path)
        except Exception as e:
            print(f"Nota: No se pudo descargar automáticamente (bloqueo de navegador): {e}")

    except FileNotFoundError:
        print(f"Error: No se encontró el archivo {input_file_path}")

# 4. Ejecución del Script
Ejecutamos el pipeline completo.
1. `setup_environment()` prepara las carpetas.
2. `analyze_logs()` ejecuta el análisis.

In [38]:
# Punto de entrada estándar de Python
if __name__ == "__main__":
    # 1. Configurar entorno y obtener la ruta del archivo de entrada
    input_file = setup_environment()

    # 2. Si el archivo existe, procedemos al análisis
    if input_file:
        analyze_logs(input_file)

--- Montando Google Drive ---
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Carpeta del proyecto encontrada: /content/drive/MyDrive/PruebaTecnicaNetec/Gemini_Logs_Project
--- 🤖 Iniciando análisis con gemini-2.5-flash ---
📄 Leyendo archivo desde: /content/drive/MyDrive/PruebaTecnicaNetec/Gemini_Logs_Project/logs.txt
📊 Volumen de datos: 80 registros detectados.
✅ [1/80] Procesado: ['Error', 'Request Processing', 'Agent']
✅ [2/80] Procesado: ['Error', 'Service', 'Restart', 'Failure']
✅ [3/80] Procesado: ['Warning', 'Session', 'Unexpected Closure']
✅ [4/80] Procesado: ['WARNING', 'Task Scheduler', 'Startup']
✅ [5/80] Procesado: ['INFO', 'Authentication', 'Token Expired', 'Credential Refresh']
✅ [6/80] Procesado: ['DEBUG', 'Database', 'Reconnection']
✅ [7/80] Procesado: ['Warning', 'Error', 'Request Processing', 'Agent']
✅ [8/80] Procesado: ['Informational', 'System', 'Scheduler', 'Startup', 'Success']
✅ [9/8

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>